In [1]:
# import required packages
# from fastai2.tabular.all import *
import xgboost as xgb
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction import DictVectorizer
from helpers import *

pd.options.display.max_columns = None

In [2]:
# path to project directory
path = Path('./')

In [3]:
# read in training dataset
train_df = pd.read_csv(path/'data/train_v7.csv', index_col=0, dtype={'season':str,
                                                                     'squad':str,
                                                                     'comp':str})

In [4]:
train_df.shape

(108515, 58)

In [5]:
# features for model
season = '2021'
cat_vars = ['season', 'position', 'was_home']
cont_vars = ['gw', 'minutes']
dep_var = ['total_points']

# find the latest gameweek
last_gw = train_df['gw'][train_df['season'] == season].max()

if np.isnan(last_gw): 
    next_gw = 1 
else: 
    next_gw = last_gw + 1
    
next_gw

30

In [6]:
# read in remaining_season.csv and sort
remaining_season_df = pd.read_csv(path/'data/remaining_season.csv', 
                                  index_col=0).sort_values(by=['gw', 'team'])

# update fields needed for processing
remaining_season_df['season'] = season
remaining_season_df['total_points'] = 0

# need to add an ordered date-like field  
remaining_season_df['kickoff_time'] = [str(9000 + x) + '-99-99' for x in remaining_season_df['match_no']]

# remove match_no, don't need it anymore
remaining_season_df = remaining_season_df.drop('match_no', axis=1)

# # append remaining season to end of training set
train_df = train_df.append(remaining_season_df, sort=False).reset_index(drop=True)

In [7]:
# # read in remaining_season.csv and sort
# remaining_season_df = pd.read_csv(path/'data/remaining_season.csv', 
#                                   index_col=0).sort_values(by=['gw', 'team'])

# remaining_season_df['season'] = season
# remaining_season_df['total_points'] = 0
# remaining_season_df['kickoff_time'] = '9999-99-99'

# # append remaining season to end of training set
# train_df = train_df.append(remaining_season_df, sort=False).reset_index(drop=True)

In [8]:
# add player lag features
lag_train_df, team_lag_vars = team_lag_features(train_df, ['total_points'], ['all', 3, 5, 10, 20])
lag_train_df, player_lag_vars = player_lag_features(lag_train_df, ['total_points'], ['all', 1, 2, 3, 5, 10, 20])

# add team conceded fields for opposition
pattern = re.compile('total_points_team_pg_last_.*_opponent')
team_lag_vars += [x.replace('team', 'team_conceded') for x in team_lag_vars if pattern.match(x)]

# but remove scored for opposition
pattern = re.compile('total_points_team_pg_last_.*_opponent')
team_lag_vars = [x for x in team_lag_vars if not pattern.match(x)]

In [9]:
team_lag_vars

['total_points_team_pg_last_all',
 'total_points_team_pg_last_3',
 'total_points_team_pg_last_5',
 'total_points_team_pg_last_10',
 'total_points_team_pg_last_20',
 'total_points_team_conceded_pg_last_all_opponent',
 'total_points_team_conceded_pg_last_3_opponent',
 'total_points_team_conceded_pg_last_5_opponent',
 'total_points_team_conceded_pg_last_10_opponent',
 'total_points_team_conceded_pg_last_20_opponent']

In [10]:
# we want to set gw and season as ordered categorical variables
# need lists with ordered categories
ordered_gws = list(range(1,39))
ordered_seasons = ['1617', '1718', '1819', '1920', '2021']

# set as categories with correct order 
lag_train_df['gw'] = lag_train_df['gw'].astype('category')
lag_train_df['season'] = lag_train_df['season'].astype('category')

lag_train_df['gw'].cat.set_categories(ordered_gws, ordered=True, inplace=True)
lag_train_df['season'].cat.set_categories(ordered_seasons, ordered=True, inplace=True)

In [11]:
lag_train_df[lag_train_df['player'] == 'Bernd Leno'].tail(25)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,assists,bonus,bps,clean_sheets,creativity,goals_conceded,goals_scored,ict_index,influence,own_goals,penalties_missed,penalties_saved,red_cards,saves,selected,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba,relative_market_value_team_season,relative_market_value_opponent_team_season,date,squad,comp,shots_total,shots_on_target,touches,pressures,tackles,interceptions,blocks,xg,npxg,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,price,short_name,total_points_team,total_points_team_conceded,total_points_team_last_all,total_points_team_conceded_last_all,total_points_team_pg_last_all,total_points_team_conceded_pg_last_all,total_points_team_last_3,total_points_team_conceded_last_3,total_points_team_pg_last_3,total_points_team_conceded_pg_last_3,total_points_team_last_5,total_points_team_conceded_last_5,total_points_team_pg_last_5,total_points_team_conceded_pg_last_5,total_points_team_last_10,total_points_team_conceded_last_10,total_points_team_pg_last_10,total_points_team_conceded_pg_last_10,total_points_team_last_20,total_points_team_conceded_last_20,total_points_team_pg_last_20,total_points_team_conceded_pg_last_20,total_points_team_opponent,total_points_team_conceded_opponent,total_points_team_last_all_opponent,total_points_team_conceded_last_all_opponent,total_points_team_pg_last_all_opponent,total_points_team_conceded_pg_last_all_opponent,total_points_team_last_3_opponent,total_points_team_conceded_last_3_opponent,total_points_team_pg_last_3_opponent,total_points_team_conceded_pg_last_3_opponent,total_points_team_last_5_opponent,total_points_team_conceded_last_5_opponent,total_points_team_pg_last_5_opponent,total_points_team_conceded_pg_last_5_opponent,total_points_team_last_10_opponent,total_points_team_conceded_last_10_opponent,total_points_team_pg_last_10_opponent,total_points_team_conceded_pg_last_10_opponent,total_points_team_last_20_opponent,total_points_team_conceded_last_20_opponent,total_points_team_pg_last_20_opponent,total_points_team_conceded_pg_last_20_opponent,minutes_last_all,minutes_last_1,minutes_last_2,minutes_last_3,minutes_last_5,minutes_last_10,minutes_last_20,total_points_last_all,total_points_pg_last_all,total_points_last_1,total_points_pg_last_1,total_points_last_2,total_points_pg_last_2,total_points_last_3,total_points_pg_last_3,total_points_last_5,total_points_pg_last_5,total_points_last_10,total_points_pg_last_10,total_points_last_20,total_points_pg_last_20
97918,Bernd Leno,14,1,90.0,Arsenal,Everton,1.317380,1.145049,False,1,0.0,0.0,10.0,0.0,0.0,2.0,0.0,0.8,8.4,0.0,0.0,0.0,0.0,1.0,577628.0,1.0,2.0,0.0,-35279.0,2232.0,37511.0,0.0,2020-12-19T17:30:00Z,2021,1.0,NaN,NaN,2020-12-19,Arsenal,Premier League,0.0,0.0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28.0,29.0,96.6,15.0,0.0,0.0,NaN,NaN,25,41.0,7357,5897.0,44.587879,35.739394,65.0,159.0,21.666667,53.000000,141.0,253.0,28.2,50.6,313.0,496.0,31.3,49.6,766.0,825.0,38.30,41.25,41.0,25.0,6835.0,6627.0,41.424242,40.163636,162.0,79.0,54.000000,26.333333,237.0,176.0,47.4,35.2,391.0,401.0,39.1,40.1,797.0,777.0,39.85,38.850000,6651.0,90.0,180.0,270.0,450.0,900.0,1170.0,265,3.585927,2.0,2.000000,4.0,2.000000,5.0,1.666667,17.0,3.400000,33.0,3.300000,45.0,3.461538
98530,Bernd Leno,15,1,90.0,Arsenal,Chelsea,1.317380,1.859440,True,8,0.0,1.0,27.0,0.0,0.0,1.0,0.0,3.6,35.6,0.0,0.0,1.0,0.0,2.0,532869.0,1.0,3.0,0.0,-44555.0,3358.0,47913.0,0.0,2020-12-26T17:30:00Z,2021,1.0,NaN,NaN,2020-12-26,Arsenal,Premier League,0.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,31.0,54.8,11.0,0.0,0.0,NaN,NaN,60,23.0,7382,5938.0,44.469880,35.771084,74.0,130.0,24.666667,43.333333,117.0,242.0,23.4,48.4,293.0,510.0,29.3,51.0,717.0,853.0,35.85,42.65,23.0,60.0,8224.0,5698.0,49.542169,34.325301,125.0,125.0,41.666667,41.666667,226.0,206.0,45.2,41.2,536.0,346.0

In [12]:
# create dataset with latest lag numbers for all future gameweeks
valid_season = '2021'
valid_gw = next_gw
valid_len = 38 - next_gw + 1

train_predict_df, train_idx, valid_idx = create_lag_train(lag_train_df, 
                                                      cat_vars, cont_vars, 
                                                      player_lag_vars, team_lag_vars, dep_var,
                                                      valid_season, valid_gw, valid_len)

In [13]:
team_lag_vars

['total_points_team_pg_last_all',
 'total_points_team_pg_last_3',
 'total_points_team_pg_last_5',
 'total_points_team_pg_last_10',
 'total_points_team_pg_last_20',
 'total_points_team_conceded_pg_last_all_opponent',
 'total_points_team_conceded_pg_last_3_opponent',
 'total_points_team_conceded_pg_last_5_opponent',
 'total_points_team_conceded_pg_last_10_opponent',
 'total_points_team_conceded_pg_last_20_opponent']

In [14]:
# take a look at the dataframe
train_predict_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114690 entries, 0 to 114689
Data columns (total 33 columns):
 #   Column                                           Non-Null Count   Dtype   
---  ------                                           --------------   -----   
 0   gw                                               114690 non-null  category
 1   minutes                                          114690 non-null  float64 
 2   minutes_last_1                                   114690 non-null  float64 
 3   minutes_last_10                                  114690 non-null  float64 
 4   minutes_last_2                                   114690 non-null  float64 
 5   minutes_last_20                                  114690 non-null  float64 
 6   minutes_last_3                                   114690 non-null  float64 
 7   minutes_last_5                                   114690 non-null  float64 
 8   minutes_last_all                                 114690 non-null  float64 
 9   oppo

In [15]:
# split out dependent variable
X, y = train_predict_df[cat_vars + cont_vars + player_lag_vars + team_lag_vars].copy(), train_predict_df[dep_var].copy()

In [16]:
# since position is categorical, it should be a string
X['position'] = X['position'].apply(str)

# need to transform season
enc = LabelEncoder()
X['season'] = enc.fit_transform(X['season'])
X_dict = X.to_dict("records")

# Create the DictVectorizer object: dv
dv = DictVectorizer(sparse=False, separator='_')

# Apply dv on df: df_encoded
X_encoded = dv.fit_transform(X_dict)

X_df = pd.DataFrame(X_encoded, columns=dv.feature_names_)

In [17]:
# split out training and prediction sets
X_train = X_df.iloc[train_idx]
y_train = y.iloc[train_idx]
X_test = X_df.iloc[valid_idx]
y_test = y.iloc[valid_idx]

In [18]:
# instatiate and train XGB Regressor
model_params = {"gamma": 0.42,
                "learning_rate": 0.047, 
                "max_depth": 4,
                "n_estimators": 171,
                "subsample": 0.6}

xg_reg = xgb.XGBRegressor(objective="reg:squarederror",
                          gamma=model_params['gamma'], 
                          learning_rate=model_params['learning_rate'], 
                          max_depth=model_params['max_depth'],
                          n_estimators=model_params['n_estimators'],
                          subsample=model_params['subsample'])

xg_reg.fit(X_train, y_train)
preds = xg_reg.predict(X_test)

In [19]:
preds

array([0.01879343, 0.00820607, 2.9296756 , ..., 0.00739357, 2.329829  ,
       0.00739357], dtype=float32)

In [20]:
remaining_season_df['predicted_points'] = preds

In [21]:
remaining_season_df.head(50)

,gw,team,opponent_team,player,position,price,play_proba,short_name,was_home,relative_market_value_team,relative_market_value_opponent_team,season,minutes,total_points,kickoff_time,predicted_points
0,30,Arsenal,Liverpool,Mesut Özil,3,6.7,0.00,Özil,True,1.280364,2.355067,2021,0.000000,0,9000-99-99,0.018793
1,30,Arsenal,Liverpool,Sokratis Papastathopoulos,2,4.8,0.00,Sokratis,True,1.280364,2.355067,2021,0.000000,0,9000-99-99,0.008206
2,30,Arsenal,Liverpool,David Luiz Moreira Marinho,2,5.4,1.00,David Luiz,True,1.280364,2.355067,2021,73.600000,0,9000-99-99,2.929676
3,30,Arsenal,Liverpool,Pierre-Emerick Aubameyang,3,11.5,1.00,Aubameyang,True,1.280364,2.355067,2021,53.400000,0,9000-99-99,1.944638
4,30,Arsenal,Liverpool,Cédric Soares,2,4.6,1.00,Cédric,True,1.280364,2.355067,2021,36.000000,0,9000-99-99,1.120056
5,30,Arsenal,Liverpool,Alexandre Lacazette,4,8.2,1.00,Lacazette,True,1.280364,2.355067,2021,60.600000,0,9000-99-99,3.067349
6,30,Arsenal,Liverpool,Shkodran Mustafi,2,5.0,0.00,Mustafi,True,1.280364,2.355067,2021,0.000000,0,9000-99-99,-0.043251
7,30,Arsenal,Liverpool,Bernd Leno,1,5.0,1.00,Leno,True,1.280364,2.355067,2021,90.000000,0,9000-99-99,3.640587
8,30,Arsenal,Liverpool,Granit Xhaka,3,5.2,1.00,Xhaka,True,1.280364,2.355067,2021,86.600000,0,9000-99-99,3.477662
9,30,Arsenal,Liverpool,Pablo Marí,2,4.4,1.00,Marí,True,1.280364,2.355067,2021,72.000000,0,9000-99-99,2.790282


In [22]:
remaining_season_df[remaining_season_df['gw'] == next_gw].sort_values(by='predicted_points', ascending=False).head(50)

,gw,team,opponent_team,player,position,price,play_proba,short_name,was_home,relative_market_value_team,relative_market_value_opponent_team,season,minutes,total_points,kickoff_time,predicted_points
209,30,Manchester United,Brighton and Hove Albion,Bruno Miguel Borges Fernandes,3,11.5,1.00,Fernandes,True,1.674080,0.579673,2021,90.000000,0,9006-99-99,6.855454
3337,30,Tottenham Hotspur,Newcastle United,Harry Kane,4,11.6,1.00,Kane,False,1.587455,0.589233,2021,87.800000,0,9007-99-99,6.475997
3339,30,Tottenham Hotspur,Newcastle United,Heung-Min Son,3,9.4,1.00,Son,False,1.587455,0.589233,2021,75.600000,0,9007-99-99,5.836723
69,30,Chelsea,West Bromwich Albion,César Azpilicueta,2,5.8,1.00,Azpilicueta,True,1.818298,0.288554,2021,90.000000,0,9002-99-99,5.784314
139,30,Leeds,Sheffield United,Patrick Bamford,4,6.7,1.00,Bamford,True,0.500510,0.356291,2021,78.800000,0,9004-99-99,5.602403
3101,30,Liverpool,Arsenal,Mohamed Salah,3,12.4,1.00,Salah,False,2.355067,1.280364,2021,84.200000,0,9000-99-99,5.534483
92,30,Chelsea,West Bromwich Albion,Edouard Mendy,1,5.2,1.00,Mendy,True,1.818298,0.288554,2021,90.000000,0,9002-99-99,5.401923
3098,30,Liverpool,Arsenal,Sadio Mané,3,11.8,1.00,Mané,False,2.355067,1.280364,2021,77.600000,0,9000-99-99,5.251654
33,30,Aston Villa,Fulham,Emiliano Martínez,1,5.4,1.00,Martínez,True,0.885365,0.520213,2021,90.000000,0,9001-99-99,5.221491
3266,30,Manchester City,Leicester City,Riyad Mahrez,3,8.1,1.00,Mahrez,False,2.401702,1.109680,2021,70.000000,0,9005-99-99,5.200767


In [23]:
# rename previous week's predictions file
# generate previous week's filename
last_gw = next_gw - 1
filename = 'history/2020-21/predictions_gw' + str(last_gw) + '.csv'
# windows
! move predictions.csv $filename
# linux
# ! mv predictions.csv $filename

        1 file(s) moved.


In [24]:
# write to predictions.csv
## RENAME PREVIOUS WEEK FIRST
predictions = remaining_season_df[['gw', 'player', 'team', 'opponent_team', 
                                   'position', 'price', 'play_proba', 'short_name',
                                   'was_home', 'predicted_points', 'minutes']]
predictions.to_csv('predictions.csv')